# 🗄️ Table Management

> Atomic table and index management utilities for tenant databases.

In [ ]:
#| default_exp utils_db

In [ ]:
#| export

from fastsql import *
from sqlalchemy import text
import logging
from typing import List, Dict, Any, Optional, Tuple, Type
from fh_saas.utils_sql import get_db_type
from nbdev.showdoc import show_doc

# Module-level logger - configured by app via configure_logging()
logger = logging.getLogger(__name__)

## 🎯 Overview

| Function | Purpose |
|----------|---------|
| `register_table` | Create table from dataclass model |
| `register_tables` | Create multiple tables atomically |
| `drop_table` | Drop table if exists |
| `create_index` | Create index with dialect-specific SQL |
| `drop_index` | Drop index if exists |
| `list_tables` | List all tables in database |
| `list_indexes` | List indexes for a table |

## 📋 Table Registration

| Function | Purpose |
|----------|---------|
| `register_table` | Create single table from dataclass |
| `register_tables` | Create multiple tables atomically |
| `drop_table` | Drop table if exists |

**Example:**

```python
class Transaction:
    id: str
    amount: float
    date: str
    category: str = None

tenant_db = get_or_create_tenant_db("tenant_123")
transactions = register_table(tenant_db, Transaction, "transactions")

# Use table object for CRUD
transactions.insert(Transaction(id="t1", amount=100.0, date="2024-01-01"))
```

In [ ]:
#| export

def register_table(tenant_db: Database, model_class: Type, table_name: str, pk: str = 'id'):
    """Create a table from a dataclass model if it doesn't exist (atomic)."""
    try:
        table = tenant_db.create(model_class, name=table_name, pk=pk)
        tenant_db.conn.commit()
        return table
    except Exception as e:
        logger.error(f"Failed to create table '{table_name}': {e}", exc_info=True)
        tenant_db.conn.rollback()
        raise Exception(f"Failed to create table '{table_name}': {str(e)}") from e

In [ ]:
show_doc(register_table)

---

### register_table

>      register_table (tenant_db:fastsql.core.Database, model_class:Type,
>                      table_name:str, pk:str='id')

*Create a table from a dataclass model if it doesn't exist (atomic).*

In [ ]:
#| export

def register_tables(tenant_db: Database, models: List[Tuple[Type, str, str]]) -> Dict[str, Any]:
    """Create multiple tables atomically (all succeed or all rollback)."""
    current_table = None
    try:
        tables = {}
        for model_class, table_name, pk in models:
            current_table = table_name
            tables[table_name] = tenant_db.create(model_class, name=table_name, pk=pk)
        tenant_db.conn.commit()
        return tables
    except Exception as e:
        logger.error(f"Failed to create table '{current_table}': {e}", exc_info=True)
        tenant_db.conn.rollback()
        raise Exception(f"Failed to create table '{current_table}': {str(e)}") from e

In [ ]:
show_doc(register_tables)

---

### register_tables

>      register_tables (tenant_db:fastsql.core.Database,
>                       models:List[Tuple[Type,str,str]])

*Create multiple tables atomically (all succeed or all rollback).*

**Parameters:**
- `models`: List of tuples `[(ModelClass, table_name, pk), ...]`

**Returns:** Dict mapping table names to table objects `{table_name: table_object}`

**Example:**

```python
class Transaction:
    id: str; amount: float; date: str

class Connection:
    id: str; provider: str; status: str

tables = register_tables(tenant_db, [
    (Transaction, "transactions", "id"),
    (Connection, "connections", "id"),
])

tables["transactions"].insert(...)
```

In [ ]:
#| export

def drop_table(tenant_db: Database, table_name: str) -> None:
    """Drop a table if it exists (atomic operation)."""
    sql = f"DROP TABLE IF EXISTS {table_name}"
    try:
        tenant_db.conn.execute(text(sql))
        tenant_db.conn.commit()
    except Exception as e:
        logger.error(f"Failed to drop table '{table_name}': {e}", exc_info=True)
        tenant_db.conn.rollback()
        raise Exception(f"Failed to drop table '{table_name}': {str(e)}") from e

In [ ]:
show_doc(drop_table)

---

### drop_table

>      drop_table (tenant_db:fastsql.core.Database, table_name:str)

*Drop a table if it exists (atomic operation).*

## 📇 Index Management

| Function | Purpose |
|----------|---------|
| `create_index` | Create index with dialect-specific SQL |
| `create_indexes` | Create multiple indexes atomically |
| `drop_index` | Drop index if exists |

**Parameters for `create_index`:**

| Parameter | Description |
|-----------|-------------|
| `table_name` | Name of the table |
| `columns` | List of column names to index |
| `unique` | If True, creates UNIQUE index (default: False) |
| `index_name` | Custom name (auto-generates `idx_{table}_{cols}` if None) |

**Example:**

```python
# Simple index
create_index(tenant_db, "transactions", ["date"])

# Composite unique index
create_index(tenant_db, "transactions", ["account_id", "external_id"], unique=True)

# Custom name
create_index(tenant_db, "transactions", ["category"], index_name="idx_txn_cat")
```

In [ ]:
#| export

def create_index(tenant_db: Database, table_name: str, columns: List[str], 
                 unique: bool = False, index_name: str = None) -> None:
    """Create an index on a table if it doesn't exist (atomic operation)."""
    # Auto-generate index name if not provided
    if index_name is None:
        index_name = f"idx_{table_name}_{'_'.join(columns)}"
    
    unique_clause = "UNIQUE " if unique else ""
    columns_clause = ", ".join(columns)
    
    # CREATE INDEX IF NOT EXISTS works for both PostgreSQL and SQLite
    sql = f"CREATE {unique_clause}INDEX IF NOT EXISTS {index_name} ON {table_name} ({columns_clause})"
    
    try:
        tenant_db.conn.execute(text(sql))
        tenant_db.conn.commit()
    except Exception as e:
        logger.error(f"Failed to create index '{index_name}' on '{table_name}': {e}", exc_info=True)
        tenant_db.conn.rollback()
        raise Exception(f"Failed to create index '{index_name}' on '{table_name}': {str(e)}") from e

In [ ]:
show_doc(create_index)

---

### create_index

>      create_index (tenant_db:fastsql.core.Database, table_name:str,
>                    columns:List[str], unique:bool=False, index_name:str=None)

*Create an index on a table if it doesn't exist (atomic operation).*

In [ ]:
#| export

def create_indexes(tenant_db: Database, indexes: List[Tuple[str, List[str], bool, Optional[str]]]) -> None:
    """Create multiple indexes atomically (all succeed or all rollback)."""
    current_index = None
    current_table = None
    try:
        for table_name, columns, unique, index_name in indexes:
            current_table = table_name
            # Auto-generate index name if not provided
            if index_name is None:
                index_name = f"idx_{table_name}_{'_'.join(columns)}"
            current_index = index_name
            
            unique_clause = "UNIQUE " if unique else ""
            columns_clause = ", ".join(columns)
            sql = f"CREATE {unique_clause}INDEX IF NOT EXISTS {index_name} ON {table_name} ({columns_clause})"
            tenant_db.conn.execute(text(sql))
        
        tenant_db.conn.commit()
    except Exception as e:
        logger.error(f"Failed to create index '{current_index}' on '{current_table}': {e}", exc_info=True)
        tenant_db.conn.rollback()
        raise Exception(f"Failed to create index '{current_index}' on '{current_table}': {str(e)}") from e

In [ ]:
show_doc(create_indexes)

---

### create_indexes

>      create_indexes (tenant_db:fastsql.core.Database,
>                      indexes:List[Tuple[str,List[str],bool,Optional[str]]])

*Create multiple indexes atomically (all succeed or all rollback).*

**Parameters:**
- `indexes`: List of tuples `[(table_name, columns, unique, index_name), ...]`
- `index_name` can be `None` for auto-generated names

**Example:**

```python
create_indexes(tenant_db, [
    ("transactions", ["date"], False, None),
    ("transactions", ["account_id", "external_id"], True, "idx_txn_unique"),
    ("connections", ["provider"], False, None),
])
```

In [ ]:
#| export

def drop_index(tenant_db: Database, index_name: str, table_name: str = None) -> None:
    """Drop an index if it exists (atomic operation)."""
    db_type = get_db_type()
    
    if db_type == "POSTGRESQL":
        # PostgreSQL: DROP INDEX IF EXISTS index_name
        sql = f"DROP INDEX IF EXISTS {index_name}"
    else:
        # SQLite: DROP INDEX IF EXISTS index_name
        sql = f"DROP INDEX IF EXISTS {index_name}"
    
    try:
        tenant_db.conn.execute(text(sql))
        tenant_db.conn.commit()
    except Exception as e:
        logger.error(f"Failed to drop index '{index_name}': {e}", exc_info=True)
        tenant_db.conn.rollback()
        raise Exception(f"Failed to drop index '{index_name}': {str(e)}") from e

In [ ]:
show_doc(drop_index)

---

### drop_index

>      drop_index (tenant_db:fastsql.core.Database, index_name:str,
>                  table_name:str=None)

*Drop an index if it exists (atomic operation).*

## 🔍 Schema Introspection

| Function | Purpose |
|----------|---------|
| `table_exists` | Check if a table exists |

**Example:**

```python
if not table_exists(tenant_db, "transactions"):
    transactions = register_table(tenant_db, Transaction, "transactions")
```

In [ ]:
#| export

def table_exists(tenant_db: Database, table_name: str) -> bool:
    """Check if a table exists in the database."""
    db_type = get_db_type()
    
    if db_type == "POSTGRESQL":
        sql = """
            SELECT EXISTS (
                SELECT FROM information_schema.tables 
                WHERE table_name = :table_name
            )
        """
    else:  # SQLite
        sql = """
            SELECT EXISTS (
                SELECT 1 FROM sqlite_master 
                WHERE type = 'table' AND name = :table_name
            )
        """
    
    try:
        result = tenant_db.conn.execute(text(sql), {"table_name": table_name})
        return result.scalar()
    except Exception as e:
        logger.error(f"Failed to check if table '{table_name}' exists: {e}", exc_info=True)
        raise Exception(f"Failed to check if table '{table_name}' exists: {str(e)}") from e

In [ ]:
show_doc(table_exists)

---

### table_exists

>      table_exists (tenant_db:fastsql.core.Database, table_name:str)

*Check if a table exists in the database.*

## Usage Example

Complete workflow: Define dataclass models → Get tenant DB → Register tables → Create indexes

In [ ]:
#| eval: false
#| hide

# Step 0: Load environment variables (required for DB connection)
from dotenv import load_dotenv
from pathlib import Path

env_path = Path().absolute() / '.env'
load_dotenv(dotenv_path=env_path)

# Step 1: Define your app-specific dataclass models
# NOTE: fastsql supports str, int, bool types. Use str for decimals/floats.
class Transaction:
    id: str
    account_id: str
    connection_id: str = None
    date: str
    amount: str              # Store as string, convert when needed
    category: str = None
    description: str = None
    created_at: str = None

class Connection:
    id: str
    provider: str          # 'plaid', 'teller', 'manual'
    external_id: str = None
    status: str = "active"
    credentials: str = None  # encrypted JSON
    last_sync: str = None
    created_at: str = None

# Step 2: Get tenant database
from fh_saas.db_tenant import get_or_create_tenant_db
tenant_db = get_or_create_tenant_db("my_tenant_123", "My App Tenant")

# Step 3: Register tables (creates if not exist, silently passes if exist)
tables = register_tables(tenant_db, [
    (Transaction, "transactions", "id"),
    (Connection, "connections", "id"),
])

# Step 4: Create indexes
create_indexes(tenant_db, [
    ("transactions", ["date"], False, None),
    ("transactions", ["account_id"], False, None),
    ("transactions", ["account_id", "connection_id"], True, "idx_txn_account_conn"),
    ("connections", ["provider"], False, None),
])

# Step 5: Use table objects for CRUD operations
transactions = tables["transactions"]
connections = tables["connections"]

# Insert a record
from fh_saas.db_host import gen_id, timestamp
new_txn = Transaction(
    id=gen_id(),
    account_id="acc_001",
    date="2024-01-15",
    amount="150.00",         # String representation of decimal
    category="groceries",
    created_at=timestamp()
)
transactions.insert(new_txn)

# Query records
all_txns = transactions()  # Get all
by_id = transactions[new_txn.id]  # Get by primary key

ℹ️  Tenant exists: My App Tenant


In [ ]:
#| eval: false
#| hide

# Test setup - load environment and get tenant DB
from dotenv import load_dotenv
from pathlib import Path

env_path = Path().absolute() / '.env'
load_dotenv(dotenv_path=env_path)

from fh_saas.db_tenant import get_or_create_tenant_db
test_db = get_or_create_tenant_db("test_utils_db", "Utils DB Test Tenant")

ℹ️  Tenant exists: Utils DB Test Tenant


In [ ]:
#| eval: false
#| hide

# Test: register_table - creates table and returns table object
class TestModel:
    id: str
    name: str
    value: str = "0.0"  # fastsql only supports str, int, bool

# Clean up first
drop_table(test_db, "test_model")
assert not table_exists(test_db, "test_model"), "Table should not exist after drop"

# Create table
test_table = register_table(test_db, TestModel, "test_model")
assert table_exists(test_db, "test_model"), "Table should exist after register"

# Insert and query
from fh_saas.db_host import gen_id
test_record = TestModel(id=gen_id(), name="test", value="42.0")
test_table.insert(test_record)
test_db.conn.commit()

retrieved = test_table[test_record.id]
assert retrieved.name == "test", f"Expected 'test', got '{retrieved.name}'"
assert retrieved.value == "42.0", f"Expected '42.0', got {retrieved.value}"

print("✅ register_table: PASSED")

✅ register_table: PASSED


In [ ]:
#| eval: false
#| hide

# Test: register_tables - bulk creation with atomic semantics
class BulkModel1:
    id: str
    field1: str

class BulkModel2:
    id: str
    field2: int = 0

# Clean up
drop_table(test_db, "bulk_model1")
drop_table(test_db, "bulk_model2")

# Bulk create
tables = register_tables(test_db, [
    (BulkModel1, "bulk_model1", "id"),
    (BulkModel2, "bulk_model2", "id"),
])

assert "bulk_model1" in tables, "bulk_model1 should be in returned tables"
assert "bulk_model2" in tables, "bulk_model2 should be in returned tables"
assert table_exists(test_db, "bulk_model1"), "bulk_model1 should exist"
assert table_exists(test_db, "bulk_model2"), "bulk_model2 should exist"

print("✅ register_tables: PASSED")

✅ register_tables: PASSED


In [ ]:
#| eval: false
#| hide

# Test: create_index and drop_index
class IndexTestModel:
    id: str
    category: str
    date: str

drop_table(test_db, "index_test")
register_table(test_db, IndexTestModel, "index_test")

# Create single index
create_index(test_db, "index_test", ["category"])
print("✅ create_index (single): PASSED")

# Create unique composite index with custom name
create_index(test_db, "index_test", ["category", "date"], unique=True, index_name="idx_custom_name")
print("✅ create_index (unique, custom name): PASSED")

# Drop index
drop_index(test_db, "idx_custom_name")
print("✅ drop_index: PASSED")

✅ create_index (single): PASSED
✅ create_index (unique, custom name): PASSED
✅ drop_index: PASSED


In [ ]:
#| eval: false
#| hide

# Test: create_indexes - bulk index creation
drop_table(test_db, "bulk_index_test")

class BulkIndexModel:
    id: str
    field_a: str
    field_b: str
    field_c: str

register_table(test_db, BulkIndexModel, "bulk_index_test")

create_indexes(test_db, [
    ("bulk_index_test", ["field_a"], False, None),
    ("bulk_index_test", ["field_b"], False, None),
    ("bulk_index_test", ["field_a", "field_c"], True, "idx_bulk_composite"),
])

print("✅ create_indexes (bulk): PASSED")

✅ create_indexes (bulk): PASSED


In [ ]:
#| eval: false
#| hide

# Test: idempotency - running register_table twice should not fail
class IdempotentModel:
    id: str
    data: str

drop_table(test_db, "idempotent_test")

# First registration
register_table(test_db, IdempotentModel, "idempotent_test")

# Second registration - should not raise error
register_table(test_db, IdempotentModel, "idempotent_test")

print("✅ idempotency (duplicate register_table): PASSED")

✅ idempotency (duplicate register_table): PASSED


In [ ]:
#| eval: false
#| hide

# Test cleanup - drop all test tables
drop_table(test_db, "test_model")
drop_table(test_db, "bulk_model1")
drop_table(test_db, "bulk_model2")
drop_table(test_db, "index_test")
drop_table(test_db, "bulk_index_test")
drop_table(test_db, "idempotent_test")

print("✅ Cleanup complete")
print("\n🎉 All tests PASSED!")

✅ Cleanup complete

🎉 All tests PASSED!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()